In [1]:
#!pip install -U tensorflow
#!pip install -U keras

import tensorflow as tf
import keras
from keras.models import load_model
import numpy as np
import h5py
import os
import pandas as pd
from pandas import DataFrame as df

from ibm_botocore.client import Config
import ibm_boto3
from IPython.display import HTML
from keras.models import model_from_json

import sys
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3


keras.__version__


Using TensorFlow backend.


'2.1.4'

# Preparing The Data
In the below cells, we load the data from IBM Cloud into our working notebook directory.
We also demonstrate IBM Watson Studio's integration with .csv files and how they can load into a pandas DataFrame seamlessly.

In [2]:
#@hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_eval = {
    'IBM_API_KEY_ID': 'Ebo3x0c7qBvPhY--ksGhQQKASTFj4OfZm1oLAHZOYU92',
    'IAM_SERVICE_ID': 'iam-ServiceId-bbeb8c1e-9ef9-41df-a342-a0e9aba1ea44',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.ng.bluemix.net/oidc/token',
    'BUCKET': 'audiosetclassification-donotdelete-pr-bochd0jdgqiuyn',
    'FILE': 'eval.h5'
}
# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_model = {
    'IBM_API_KEY_ID': 'Ebo3x0c7qBvPhY--ksGhQQKASTFj4OfZm1oLAHZOYU92',
    'IAM_SERVICE_ID': 'iam-ServiceId-bbeb8c1e-9ef9-41df-a342-a0e9aba1ea44',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.ng.bluemix.net/oidc/token',
    'BUCKET': 'audiosetclassification-donotdelete-pr-bochd0jdgqiuyn',
    'FILE': 'final_weights.h5'
}
def download_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.download_file(Bucket=credentials['BUCKET'],Key=key,Filename=local_file_name)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Downloaded')

In [3]:
#Downloading the files from IBM Cloud to the current directory
download_file_cos(credentials_model,'final_weights.h5','final_weights.h5')
download_file_cos(credentials_model,'model_final.json','model_final.json')
download_file_cos(credentials_eval,'eval.h5','eval.h5')

File Downloaded
File Downloaded
File Downloaded


In [4]:
#Loading the label indices and YouTube video ID CSV files to Pandas DataFrames
def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_d249a8db1de240d283a16bd3ddeac094 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Ebo3x0c7qBvPhY--ksGhQQKASTFj4OfZm1oLAHZOYU92',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_d249a8db1de240d283a16bd3ddeac094.get_object(Bucket='audiosetclassification-donotdelete-pr-bochd0jdgqiuyn',Key='class_labels_indices.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()
body = client_d249a8db1de240d283a16bd3ddeac094.get_object(Bucket='audiosetclassification-donotdelete-pr-bochd0jdgqiuyn',Key='eval_segments.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_2 = pd.read_csv(body)
df_data_2.head()



,video_id,start_time,end_time
0,--4gqARaEJE,0.0,10.0
1,--BfvyPmVMo,20.0,30.0
2,--U7joUcTCo,0.0,10.0
3,--i-y1v8Hy8,0.0,9.0
4,-0BIyqJj9ZU,30.0,40.0


In [5]:
#Loading the evaluation data and labels to numpy objects
def load_data(hdf5_path):
    """
    Loads the data into numpy objects. 
    Input : Path to data.
    Output : Train/Test examples, corresponding labels, corresponding youtube video_id.
    """
    with h5py.File(hdf5_path, 'r') as hf:
        x = hf.get('x')
        y = hf.get('y')
        video_id_list = hf.get('video_id_list')
        x = np.array(x)
        y = list(y)
        video_id_list = list(video_id_list)
        
    return x, y, video_id_list

def uint8_to_float32(x):
    return (np.float32(x) - 128.) / 128.
    
def bool_to_float32(y):
    return np.float32(y)

(x, y, video_id_list) = load_data('eval.h5')
x = uint8_to_float32(x)		# shape: (N, 10, 128)
y = bool_to_float32(y)		# shape: (N, 527)


# Building the Model
In the below cells we build the model and load the weights pre-trained on IBM cloud/dlaas.

In [6]:
#Building the model

import keras
from keras.models import Model
from keras.layers import (Input, Dense, BatchNormalization, Dropout, Lambda,
                          Activation, Concatenate)
import keras.backend as K
from keras.optimizers import Adam

try:
    import cPickle
except BaseException:
    import _pickle as cPickle


def average_pooling(inputs, **kwargs):
    input = inputs[0]   # (batch_size, time_steps, freq_bins)
    return K.mean(input, axis=1)


def max_pooling(inputs, **kwargs):
    input = inputs[0]   # (batch_size, time_steps, freq_bins)
    return K.max(input, axis=1)


def attention_pooling(inputs, **kwargs):
    [out, att] = inputs

    epsilon = 1e-7
    att = K.clip(att, epsilon, 1. - epsilon)
    normalized_att = att / K.sum(att, axis=1)[:, None, :]

    return K.sum(out * normalized_att, axis=1)


def pooling_shape(input_shape):

    if isinstance(input_shape, list):
        (sample_num, time_steps, freq_bins) = input_shape[0]

    else:
        (sample_num, time_steps, freq_bins) = input_shape

    return (sample_num, freq_bins)

model_type = 'decision_level_multi_attention'
time_steps = 10
freq_bins = 128
classes_num = 527

# Hyper parameters
hidden_units = 1024
drop_rate = 0.5
batch_size = 500

# Embedded layers
input_layer = Input(shape=(time_steps, freq_bins))

a1 = Dense(hidden_units)(input_layer)
a1 = BatchNormalization()(a1)
a1 = Activation('relu')(a1)
a1 = Dropout(drop_rate)(a1)

a2 = Dense(hidden_units)(a1)
a2 = BatchNormalization()(a2)
a2 = Activation('relu')(a2)
a2 = Dropout(drop_rate)(a2)

a3 = Dense(hidden_units)(a2)
a3 = BatchNormalization()(a3)
a3 = Activation('relu')(a3)
a3 = Dropout(drop_rate)(a3)

# Pooling layers
if model_type == 'decision_level_max_pooling':
    '''Global max pooling.

    [1] Choi, Keunwoo, et al. "Automatic tagging using deep convolutional 
    neural networks." arXiv preprint arXiv:1606.00298 (2016).
    '''
    cla = Dense(classes_num, activation='sigmoid')(a3)
    output_layer = Lambda(max_pooling, output_shape=pooling_shape)([cla])

elif model_type == 'decision_level_average_pooling':
    '''Global average pooling.

    [2] Lin, Min, et al. Qiang Chen, and Shuicheng Yan. "Network in 
    network." arXiv preprint arXiv:1312.4400 (2013).
    '''
    cla = Dense(classes_num, activation='sigmoid')(a3)
    output_layer = Lambda(
        average_pooling,
        output_shape=pooling_shape)(
        [cla])

elif model_type == 'decision_level_single_attention':
    '''Decision level single attention pooling.

    [3] Kong, Qiuqiang, et al. "Audio Set classification with attention
    model: A probabilistic perspective." arXiv preprint arXiv:1711.00927
    (2017).
    '''
    cla = Dense(classes_num, activation='sigmoid')(a3)
    att = Dense(classes_num, activation='softmax')(a3)
    output_layer = Lambda(
        attention_pooling, output_shape=pooling_shape)([cla, att])

elif model_type == 'decision_level_multi_attention':
    '''Decision level multi attention pooling.

    [4] Yu, Changsong, et al. "Multi-level Attention Model for Weakly
    Supervised Audio Classification." arXiv preprint arXiv:1803.02353
    (2018).
    '''
    cla1 = Dense(classes_num, activation='sigmoid')(a2)
    att1 = Dense(classes_num, activation='softmax')(a2)
    out1 = Lambda(
        attention_pooling, output_shape=pooling_shape)([cla1, att1])

    cla2 = Dense(classes_num, activation='sigmoid')(a3)
    att2 = Dense(classes_num, activation='softmax')(a3)
    out2 = Lambda(
        attention_pooling, output_shape=pooling_shape)([cla2, att2])

    b1 = Concatenate(axis=-1)([out1, out2])
    b1 = Dense(classes_num)(b1)
    output_layer = Activation('sigmoid')(b1)

elif model_type == 'feature_level_attention':
    '''Feature level attention.

    [1] To be appear.
    '''
    cla = Dense(hidden_units, activation='linear')(a3)
    att = Dense(hidden_units, activation='sigmoid')(a3)
    b1 = Lambda(
        attention_pooling, output_shape=pooling_shape)([cla, att])

    b1 = BatchNormalization()(b1)
    b1 = Activation(activation='relu')(b1)
    b1 = Dropout(drop_rate)(b1)

    output_layer = Dense(classes_num, activation='sigmoid')(b1)

else:
    raise Exception("Incorrect model_type!")

# Build model
model = Model(inputs=input_layer, outputs=output_layer)
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 10, 128)      0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 10, 1024)     132096      input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 10, 1024)     4096        dense_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 10, 1024)     0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
dropout_1 

In [7]:
#Loading weights
model.load_weights("final_weights.h5")

## Demo supporting code 
This is the supporting code to run the demo at the end of this notebook.

In [8]:
def demo(v):
    """
    A function to demonstrate the audio classification.
    Input : a random number to retrieve a query video.
    Output : video embed string for the YouTube video.
    Prints : Top 5 class probabilities from the classifier.
    """
    test_data = x[v:v+1,:,:]
    current_infer = model.predict(test_data)
    current_video = str(video_id_list[v],'utf-8')
    start_time = int(df_data_2.loc[df_data_2['video_id'] == current_video]['start_time'])
    video_string = '<iframe width="560" height="315" src="https://www.youtube.com/embed/'+current_video+'?autoplay=1&start='+str(start_time)+';autoplay=1 frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>'
    print('Predicted Labels :\n')
    print(df_data_1.loc[current_infer[0].argsort()[-5:][::-1]]['display_name'])
    #print('Ground Truth Labels :\n')
    #print(df_data_1.loc[y[v].argsort()[-5:][::-1]]['display_name'])
    print("Playing video snippet below.. Turn on the speakers..\n")
    return video_string

# Real Time Demo

Replace the number below to change the video and generate a real-time inference.

In [9]:
#Try out a random number between 0 and 20000 
video_number = 9384
HTML(demo(video_number))

Predicted Labels :

401               Foghorn
137                 Music
152    Keyboard (musical)
186           French horn
153                 Piano
Name: display_name, dtype: object
Playing video snippet below.. Turn on the speakers..



# Reverse Search Audio using keywords

In this section we will first perform inference on the full test dataset and store it to later retrieve search results.

In [10]:
#First we get the inference for all our data and store it under `inferences`
inferences = model.predict(x[0:1000])

In [11]:
#Now we write a function to retrieve the required audios
def search_audio(keyword, infer=inferences):
    """
    Uses the inferences made using the audioset classifier to lookup and retrieve relevant audio/video for a given keyword.
    Inputs : search query, inferences on test dataset.
    Outputs : YouTube embed link for the strongest video returned and a list of all videos containing the search query.
    """
    try:
        class_id = int(df_data_1.loc[df_data_1['display_name'] == keyword]['index'])
        print("Retrieving videos containing "+keyword +" with the highest probabilities...")
    except:
        class_id = -1
        print("Search Query is invalid!")
    strong_inferences = np.argsort(inferences,axis = 1)[:,-5:]
    video_ids = np.where(strong_inferences==class_id)[0]
    video_strings = []
    max_prob = -1
    strongest_video_embed = ''
    print_counter = 0
    if(video_ids.any()):
        print("...Done!")
        print("Printing a few strong results.. ")
    for v in video_ids:
        current_video = str(video_id_list[v],'utf-8')
        start_time = int(df_data_2.loc[df_data_2['video_id'] == current_video]['start_time'])
        video_string = 'https://www.youtube.com/watch?v='+current_video+'&t='+str(start_time)
        if(inferences[v][class_id]>0.8 and print_counter<10):
            print(video_string + " contains "+keyword+" with probability "+ str(inferences[v][class_id]))
            print_counter +=1
        video_strings.append(video_string)
        if inferences[v][class_id]>max_prob:
            max_prob = inferences[v][class_id]
            max_video = current_video
            max_time = start_time
    if(not video_strings):
        print("No videos found for the search query: "+keyword )
    else:
        strongest_video_embed = '<iframe width="560" height="315" src="https://www.youtube.com/embed/'+max_video+'?autoplay=1&start='+str(max_time)+';autoplay=1 frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>'
    return strongest_video_embed,video_strings

In [12]:
search_query = 'Car'
embed = search_audio(search_query)[0]
if(embed):
    print("\n Playing video with the strongest probability of containing "+search_query+" ..")
HTML(embed)

Retrieving videos containing Car with the highest probabilities...
...Done!
Printing a few strong results.. 
https://www.youtube.com/watch?v=-CZ1LIc8aos&t=20 contains Car with probability 0.863814
https://www.youtube.com/watch?v=-Wd5YV97ftU&t=320 contains Car with probability 0.948551
https://www.youtube.com/watch?v=1G6wtnmdZhE&t=30 contains Car with probability 0.893657

 Playing video with the strongest probability of containing Car ..
